# Multiclass Classification

As convenient as it would be, most things in the world are not binary prediction problems. Many are multiclass prediction problems: given an input, map it to one of 10 classes, or 20 classes, or 100k classes.

# 20 Newsgroups

As a running example, we'll use the 20 Newsgroups dataset. This contains about 20,000 postings to 20 different newsgroups on usenet; the classification task is to put a posting into the proper newsgroup. The set of newsgroups are:

    comp.graphics       comp.os.ms-windows.misc  comp.sys.ibm.pc.hardware  comp.sys.mac.hardware  comp.windows.x
    talk.politics.misc  talk.politics.guns       talk.politics.mideast     talk.religion.misc
    rec.autos           rec.motorcycles          rec.sport.baseball        rec.sport.hockey
    sci.crypt           sci.electronics          sci.med                   sci.space
    alt.atheism         soc.religion.christian
    misc.forsale

which we first need to download and extract. Thankfully, [Jason Rennie has a nice distribution](http://qwone.com/~jason/20Newsgroups/); this is a 14mb download.

In [ ]:
!rm -rf data/20news-*
!curl -o data/20news-bydate.tar.gz http://qwone.com/~jason/20Newsgroups/20news-bydate.tar.gz
!tar zxC data -f data/20news-bydate.tar.gz
!echo ""
!echo "List of directories:"
!echo ""
!ls data/20news-bydate-train/
!echo ""
!echo "First training document in comp.windows.x"
!echo ""
!head -n40 data/20news-bydate-train/comp.windows.x/64830

As you're probably used to, our first step is to get this into `vw` format.

Although `vw` does support labels with names, we're going to map the labels to numbers. We'll come back to named labels later.

For features, we'll extract the different email headers to different namespaces, and then the body of the message to a separate namespace. We'll do very stupid tokenization; I'm sure you could do better if you wanted.

Here's some code to generate the relevant `vw` files:

In [ ]:
import re, os
from __future__ import print_function
import io

def sanify(s): return s.replace(':',';').replace('|','/')   # replace : with ; and | with /
def tokenize(s): return re.sub('([^A-Za-z0-9 ]+)', ' \\1 ', s).split()  # add space around anything not alphanum

headerToNamespace = {'Subject': 'S',
                     'From': 'F',
                     'Organization': 'O',
                     'Distribution': 'D',
                     'Reply-To': 'R',
                     'Keywords': 'K',
                     'Summary': 'U' }

def readSinglePost(filename):
    with io.open(filename, 'r', encoding='iso-8859-1') as h:
        namespaces = {}
        text = []
        inText = False
        for l in h.readlines():
            words = tokenize(l.strip())
            if not inText and len(words) == 0:
                inText = True
            elif inText:
                text += words
            elif len(words) > 2 and words[1] == ':' and words[0] in headerToNamespace:
                ns = headerToNamespace[words[0]]
                namespaces[ns] = words[2:]
        namespaces['w'] = text
        return namespaces

def postToVW(label, namespaces):
    ex = str(label)
    for ns,words in namespaces.items():
        ex += ' |%s %s' % (ns, ' '.join(map(sanify,words[:5000])))  # only take the first 5k words of a post
    return ex

def readPostsInDirectory(directory):
    return [readSinglePost(directory + os.sep + f)
            for f in os.listdir(directory)
            if  f.isdigit()]   # all file names are just numbers

def readAllPosts(baseDirectory, newsgroups):
    examples = []
    for name,label in newsgroups.items():
        posts = readPostsInDirectory(baseDirectory + os.sep + name)
        examples += [postToVW(label, post) for post in posts]
    return examples

def writeFile(filename, examples):
    with io.open(filename,'w', encoding='iso-8859-1') as h:
        for ex in examples:
            print("{0}".format(ex), file=h)

newsgroupNames = 'comp.graphics comp.os.ms-windows.misc comp.sys.ibm.pc.hardware comp.sys.mac.hardware comp.windows.x talk.politics.misc talk.politics.guns talk.politics.mideast talk.religion.misc rec.autos rec.motorcycles rec.sport.baseball rec.sport.hockey sci.crypt sci.electronics sci.med sci.space alt.atheism soc.religion.christian misc.forsale'.split()
newsgroups = { name: k+1 for k,name in enumerate(newsgroupNames) }   # label ids have to start at 1, not 0

train = readAllPosts('data/20news-bydate-train', newsgroups)
test  = readAllPosts('data/20news-bydate-test',  newsgroups)

import random
random.seed(9876)
random.shuffle(train)

print('read {0} training examples and {1} test examples'.format(len(train), len(test)))

writeFile('data/20ng.tr', train)
writeFile('data/20ng.te', test)

print('files generated:')
!wc data/20ng.tr data/20ng.te

# <a id="train"></a> Training VW using One Against All

We now have to train `vw`. Basically the only difference between a multiclass run of `vw` and a binary classification run is that you have to tell `vw` how you want it to solve the multiclass problem, and how many classes there are.

A very reasonable method for multiclass classification is **One Against All** (OAA), which, on $K$ classes, trains $K$ regressors, each to predict a particular class. We can do this for 20 Newsgroups by simply saying "`--oaa 20`" on the command line.

In [ ]:
!vw -k -c -b 27 --oaa 20 -d data/20ng.tr -f data/20ng.model --passes 20 --holdout_after 10001

At the end, we have a holdout loss of about 9.0% -- this is just the fraction of errors we make. This is pretty reasonable; [Jason Rennie reports 15% error](http://people.csail.mit.edu/jrennie/writing/loocv.pdf) also on the test data. But we expect test performance should be similar to heldout performance. Let's make sure:

In [ ]:
!vw -t -i data/20ng.model -d data/20ng.te

Okay, so 19% error is actually much worse than 9% error. This suggests the training and test distributions don't match up entirely. Oh well. Still ballpark reasonable for putting no effort into it.

A few notes:

* When I trained the model, I used a slightly larger number of bits than before. This is because in OAA, we're storing 20 different weight vectors in the same amount of space. If you want each weight vector to have the, say, 24 bits of representation, then you should run with 28 or 29 total bits to account for the fact that you need to pack in 20 different vectors.

* When I tested the model I didn't need to say `--oaa` again; this is stored in the saved model and therefore unnecessary.

# <a id="human"></a> Getting a Human Readable Model

The mechanism for getting a human-readable model out is basically the same as always. We train a model, then rerun with `--invert_hash -t` over the training data.

In [ ]:
!vw -t -i data/20ng.model -d data/20ng.tr --invert_hash data/20ng.model.readable

Unfortunately, this is quite slow and memory-hogging on my laptop (about 1gb ram, a few minutes). On the bright side, this *does* demonstrate that there's a big win for now dealing with strings during normal training!

Once that's done, we can look at specific features. For instance, we might be interested in the feature "DoD" (probably "Department of Defense") in the "w" (words) namespace:

In [ ]:
!grep '^w^bike\[' data/20ng.model.readable

The way to read this is as follows. For the `w` namespace, there's a feature `bike`. It has a separate weight for each of the classes `0..19` (this is potentially an annoying source of off-by-one erorrs since in the input file we have classes `1..20`... sigh, sorry!).

The class that `bike` is most indicative of is vw label `[10]` (with a weight of 0.267542). This is our class 11. Which class is that?

In [ ]:
for k, v in newsgroups.items():
    if v == 11:
        print(k)

Okay, so it's `rec.motorcycles`. This passes the sanity check!

We can also just look at the top weighted features:

In [ ]:
!cat data/20ng.model.readable | tail -n+14 | sort -t: -k3g | tail

The top two are the `w`ord namespace, but the third one (`S^Windows`) is the Subject line namespace. In this case it's indicative of label `[1]` which is class 2, which, looking above, is `comp.os-ms-windows.misc`.

# <a id="raw"> Getting (Raw) Predictions

As before, often we want to get label predictions (which class?) and/or per-label scores (a ranking?). We can get the class predictions with `-p` and the score with `-r`. Let's do this for the test data.

In [ ]:
!vw -t -i data/20ng.model -d data/20ng.te -p data/20ng.te.pred -r data/20ng.te.raw --quiet
!head data/20ng.te.pred data/20ng.te.raw

In the `.pred` file, we can see the actual predictions (20, 20, 20, ..., 3, 20, 20, ...). These are just the predicted labels.

In the `.raw` file, we can see per-class scores. This is still one-line-per-example. For the first line, we can see that every class 1..19 has a negative score and 20 has a positive score. Thus 20 "wins" and is the prediction.

# <a id="namd"></a> Using Named Classes

Sometimes it is inconvenient to have to map class ids to numbers. Using "named classes" we can avoid this. The downside is that the class names have to, instead, be specified on the command line.

<font size="-2">[Note: you might wonder why `vw` can't just read in named classes as it reads the file and do it's own internal counting of numbers. This is because sometimes you want to run multiple instances of `vw` in parallel. When these instances communicate, they need to agree on the numbering scheme.]</font>

To use named classes, you simply: (1) replace the label numbers in the input file(s) with strings; and (2) specify the names on the command line. Here's an example for 20 Newsgroups.

In [ ]:
namedNewsgroups = { name:name for name in newsgroupNames }

train = readAllPosts('data/20news-bydate-train', namedNewsgroups)
test  = readAllPosts('data/20news-bydate-test',  namedNewsgroups)

random.seed(9876)
random.shuffle(train)

print('read {0} training examples and {1} test examples\n'.format(len(train), len(test)))
print('all labels: {0}\n'.format(','.join(newsgroups)))

writeFile('data/20ng-named.tr', train)
writeFile('data/20ng-named.te', test)

print('files generated:')
!wc data/20ng-named.tr data/20ng-named.te
print('\nsome of the examples 9first few words:')
!head data/20ng-named.tr | cut -d' ' -f1-10
print('\ntraining vw:')
!vw -k -c -b 27 --oaa 20 -d data/20ng-named.tr -f data/20ng-named.model --passes 20 --holdout_after 10001 --named_labels misc.forsale,talk.politics.guns,comp.sys.mac.hardware,talk.politics.misc,soc.religion.christian,comp.graphics,sci.med,talk.religion.misc,comp.windows.x,comp.sys.ibm.pc.hardware,talk.politics.mideast,comp.os.ms-windows.misc,sci.crypt,sci.space,alt.atheism,rec.sport.hockey,rec.sport.baseball,sci.electronics,rec.autos,rec.motorcycles

As before, we might want to get (raw) predictions out:

In [ ]:
!vw -t -i data/20ng-named.model -d data/20ng-named.te -p data/20ng-named.te.pred -r data/20ng-named.te.raw --quiet
!head data/20ng-named.te.pred data/20ng-named.te.raw

Here, you can see that the predictions shows the name of the class. The raw predictions, however, uses the internal numbering. The question then is "what labels do the different numbers correspond to?" The answer is that it's just the order you used in the "`--named_labels`" argument. In that argument, the first item was `misc.forsale` so that's what `1:...` means here.



# <a id="scaling"></a> Scaling Up to More Classes

In the previous example, we had only 20 classes. For lots of problems, we have more. The problem we'll consider now is the Quiz Bowl question answering task. In this problem, we get a long-form question and have to provide an answer. In the full version of the problem, the goal is to answer the question as *early* as possible (i.e., before the entire question is read). For simplicity here, we'll focus on the simpler problem of just predicting the answer at the end of the question. Our [QANTA system](), which [recently bested Ken Jennings](), get an error rate of about 20% at the end of the question on this task (though trained on more data and also with extra Wikipedia resources). [Mohit Iyyer]() has made [the non-proprietary question/answer pairs available for download]().

As usual, we begin by grabbing the data (this is a 31mb download):

In [ ]:
!rm -rf data/question-* data/quizbowl*
!curl -o data/question_data.tar.gz http://cs.umd.edu/~miyyer/data/question_data.tar.gz
!tar zxC data -f data/question_data.tar.gz
!echo ""
!echo "counting the number of questions:"
!wc -l data/question_data/questions.csv
!echo ""
!echo "looking at the first question:"
!echo ""
!head -n2 data/question_data/questions.csv

(In this format, "`|||`" means "sentence boundary".)

And now we have to read it in. In this case it's particularly easy because it's generated with Python's csvwriter. The data is already split into train/dev/test for us too:

In [ ]:
import csv
import sys

def readQuizBowlData(filename):
    train,dev,test = [],[],[]
    data = csv.reader(io.open(filename, 'r', encoding='iso-8859-1').readlines())
    header = next(data, None)
    if set(header) != set(['Question ID', 'Fold', 'Category', 'Answer', 'Text']):
        raise Exception('data improperly formatted')
    for item in iter(data):
        y = item[3]        
        x = sanify(' '.join(tokenize(item[4].replace('|||',''))))
        if   item[1] == 'train': train.append( (x,y) )
        elif item[1] == 'dev'  :   dev.append( (x,y) )
        elif item[1] == 'test' :  test.append( (x,y) )
    return train,dev,test

def makeLabelIDs(train, outputFile):
    labelIds = { label: k+1 for k,label in enumerate(set([y for x,y in train])) }
    labelIds['***UNKNOWN***'] = len(labelIds)+1
    with io.open(outputFile, 'w') as h:
        for label,k in labelIds.items():
            print("{0}\t{1}".format(k, label), file=h)
    return labelIds

def writeVWFile(filename, data, labelIds):
    unknownId = labelIds['***UNKNOWN***']
    with io.open(filename,'w') as h:
        for x,y in data:
            print("{0} | q {1}".format(labelIds.get(y, unknownId), x), file=h)
            
train,dev,test = readQuizBowlData('data/question_data/questions.csv')
traindev = train + dev
random.seed(9876)
random.shuffle(traindev)
labelIds = makeLabelIDs(train, 'data/quizbowl.labels')
writeVWFile('data/quizbowl.trde', traindev, labelIds)
writeVWFile('data/quizbowl.te',   test    , labelIds)
print('maximum label id = {0}'.format(len(labelIds)+1))
!wc data/quizbowl.t[re]*

So we have 2322 labels and 17805 train/dev examples. This is obviously going to be a pretty hard problem without auxiliary information. We can still make a go of it!

In [ ]:
!vw -k -c -b 27 -d data/quizbowl.trde --oaa 2322 --passes 20 -f data/quizbowl.model

That's 27% holdout error, which, all told, is not that bad. Of course the real question is how well we do on test:

In [ ]:
!vw -d data/quizbowl.te -t -i data/quizbowl.model

Wow, 20% test loss. This is actually quite impressive for basically doing no work. One legitimate question is what is the frequency of the most frequent class:

In [ ]:
from collections import Counter
labelFreq = Counter([y for x,y in train])
print(labelFreq.most_common(10))

So out of 8800 training examples, the most frequent one only occurs twenty times, which is not very much. Of course the "unknown" label could dominate in test. It's unlikely we would predict that label since we never saw it at training time, but it's possible. Let's make sure:

In [ ]:
!cat data/quizbowl.te | cut -d' ' -f1 | sort | uniq -c | sort -g | tail

So the most frequent label *is* indeed the "never seen this answer before" label, but it still only appears in 75/2600 examples, which is only about 2.8%. Definitely not a winning strategy!

# <a id="alter"></a> Alternatives to One Against All

You may have noticed that training on the quizbowl data, especially for a relatively small data set, was kind of slow. This is because OAA time complexity scales like O(K), where K is the number of classes. This is fine for something like 20 Newsgroups, but bad for problems with lots of classes.



In [ ]:
!time vw -k -c -b 29 -d data/quizbowl.trde --log_multi 2322 --passes 10 --early_terminate 999 -f data/quizbowl.model2
!vw -d data/quizbowl.te -t -i data/quizbowl.model2 2>&1 | grep '^average loss'

So that took (on my laptop) about 2 minutes of wallclock time and gets an error rate of 40%, which is significantly worse than OAA. If we give OAA approximately the same amount of time, this is what happens:

In [ ]:
!time vw -k -c -b 27 -d data/quizbowl.trde --oaa 2322 --passes 2 -f data/quizbowl.model
!vw -d data/quizbowl.te -t -i data/quizbowl.model2 2>&1 | grep '^average loss'

So that took about the same amount of time and got a loss of 40%.

In this case, it appears there is a flat out tie :(. However, as the number of classes grows, eventually OAA becomes untenable and you really cannot afford the O(K) computation. More details are in the [LOMTree paper](http://arxiv.org/abs/1406.1822).